In [2]:
import pandas as pd
data = pd.read_csv('./chinese_news.csv')
data.head()

,date,tag,headline,content
0,2016-01-01,详细全文,陆军领导机构火箭军战略支援部队成立大会在京举行 习近平向中国人民解放军陆军火箭军战略支援部队...,中国人民解放军陆军领导机构、中国人民解放军火箭军、中国人民解放军战略支援部队成立大会2015...
1,2016-01-01,详细全文,中央军委印发《关于深化国防和军队改革的意见》,经中央军委主席习近平批准，中央军委近日印发了《关于深化国防和军队改革的意见》。\n《意见》强...
2,2016-01-01,详细全文,《习近平关于严明党的纪律和规矩论述摘编》出版发行,由中共中央纪律检查委员会、中共中央文献研究室编辑的《习近平关于严明党的纪律和规矩论述摘编》一...
3,2016-01-01,详细全文,以实际行动向党中央看齐 向高标准努力,广大党员干部正在积极学习习近平总书记在中央政治局专题民主生活会上的重要讲话。大家纷纷表示要把...
4,2016-01-01,详细全文,【年终特稿】关键之年 改革挺进深水区,刚刚过去的2015年，是全面深化改革的关键之年。改革集中发力在制约经济社会发展的深层次矛盾，...


In [3]:
import re
import jieba
import cntext as ct

stopwords = ct.load_pkl_dict('STOPWORDS.pkl')['STOPWORDS']['chinese']

def clean_text(text):
    words = jieba.lcut(text)
    words = [w for w in words if w not in stopwords]
    return ' '.join(words)

data['head_div']=data['headline'].apply(clean_text)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/b_/lh9rsw9n4j100xyqrk8sdbk40000gn/T/jieba.cache
Loading model cost 0.530 seconds.
Prefix dict has been built successfully.


In [1]:
data_totiao = set()
allow_topic = ['102','103','110','109']
with open('./toutiao_cat_data.txt','r',encoding='utf-8') as file:
    for line in file:
        parts = line.strip().split('_!_')
        if (len(parts)>=4 ):
            data_totiao.add((parts[3],parts[1]))

In [2]:
print(len(data_totiao))

296493


In [5]:
data_totiao=list(data_totiao)[0:5000]
data_totiao_div = []
for i in data_totiao:
    data_totiao_div.append(clean_text(i[0]))


In [6]:
from gensim.models import KeyedVectors
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.cluster import KMeans
from bertopic import BERTopic

umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')
hdbscan_model = HDBSCAN(min_cluster_size=300, metric='euclidean', cluster_selection_method='eom')
cluster_model = KMeans(n_clusters=4)

In [7]:
word2vec_list=['sgns.wiki.word.bz2','sgns.financial.word.bz2','sgns.literature.word.bz2','sgns.merge.word.bz2','sgns.renmin.word.bz2','sgns.sogou.word.bz2',
               'sgns.target.word-word.dynwin5.thr10.neg5.dim300.iter5.bz2','sgns.zhihu.word.bz2']
for word2vec_model in word2vec_list:

    chinese_news_models = KeyedVectors.load_word2vec_format(word2vec_model, unicode_errors='ignore')
    topic_model = BERTopic(language="chinese (simplified)", 
                       embedding_model=chinese_news_models,
                       umap_model=umap_model,
                       hdbscan_model=hdbscan_model,
                       calculate_probabilities=False, 
                       verbose=True,
                       )
    
    # docs = data['head_div'].tolist()
    docs = data_totiao_div
    #2000条进行fit_transform需要1min
    topics, probs = topic_model.fit_transform(docs)
    new_topics = topic_model.reduce_outliers(docs, topics)
    topic_model.update_topics(docs, topics=new_topics)
    # ---
    topic_msg=topic_model.get_document_info(docs)
    topic_div=topic_msg['Topic'].tolist()
    ori_topic = []
    for i in data_totiao:
        ori_topic.append(i[1])
    # ['102','103','110','109']
    acc_num=0
    for i in range(len(ori_topic)):
        a=topic_div[i]
        b=ori_topic[i]
        if (a==0 and b=='102') or (a==1 and b=='109') or (a==2 and b=='103') or (a==3 and b=='110'):
            acc_num+=1
    print('---')
    print(word2vec_model)
    print(acc_num/len(ori_topic))
    print('---')




2023-12-23 17:32:56,136 - BERTopic - Embedding - Transforming documents to embeddings.
100%|██████████| 5000/5000 [00:00<00:00, 20573.61it/s]
2023-12-23 17:32:56,424 - BERTopic - Embedding - Completed ✓
2023-12-23 17:32:56,425 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2023-12-23 17:33:18,420 - BERTopic - Dimensionality - Completed ✓
2023-12-23 17:33:18,421 - BERTopic - Cluster - Start clustering the reduced embeddings
2023-12-23 17:33:18,772 - BERTopic - Cluster - Completed ✓
2023-12-23 17:33:18,780 - BERTopic - Representation - Extracting topics from clusters using representation models.
2023-12-23 17:33:18,880 - BERTopic - Representation - Completed ✓
100%|██████████| 1/1 [00:00<00:00, 94.58it/s]
2023-12-23 17:33:18,933 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings

---
sgns.wiki.word.bz2
0.2496
---


2023-12-23 17:35:31,030 - BERTopic - Embedding - Transforming documents to embeddings.
100%|██████████| 5000/5000 [00:00<00:00, 22694.80it/s]
2023-12-23 17:35:31,275 - BERTopic - Embedding - Completed ✓
2023-12-23 17:35:31,276 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2023-12-23 17:35:35,931 - BERTopic - Dimensionality - Completed ✓
2023-12-23 17:35:35,932 - BERTopic - Cluster - Start clustering the reduced embeddings
2023-12-23 17:35:36,259 - BERTopic - Cluster - Completed ✓
2023-12-23 17:35:36,262 - BERTopic - Representation - Extracting topics from clusters using representation models.
2023-12-23 17:35:36,364 - BERTopic - Representation - Completed ✓
100%|██████████| 2/2 [00:00<00:00, 41.36it/s]
2023-12-23 17:35:36,457 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings

---
sgns.financial.word.bz2
0.5368
---


2023-12-23 17:36:35,401 - BERTopic - Embedding - Transforming documents to embeddings.
100%|██████████| 5000/5000 [00:00<00:00, 24754.80it/s]
2023-12-23 17:36:35,616 - BERTopic - Embedding - Completed ✓
2023-12-23 17:36:35,616 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2023-12-23 17:36:40,092 - BERTopic - Dimensionality - Completed ✓
2023-12-23 17:36:40,093 - BERTopic - Cluster - Start clustering the reduced embeddings
2023-12-23 17:36:40,456 - BERTopic - Cluster - Completed ✓
2023-12-23 17:36:40,460 - BERTopic - Representation - Extracting topics from clusters using representation models.
2023-12-23 17:36:40,561 - BERTopic - Representation - Completed ✓
100%|██████████| 2/2 [00:00<00:00, 38.47it/s]
2023-12-23 17:36:40,653 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings

...

2023-12-23 17:43:27,764 - BERTopic - Embedding - Transforming documents to embeddings.
100%|██████████| 5000/5000 [00:00<00:00, 12200.78it/s]
2023-12-23 17:43:29,653 - BERTopic - Embedding - Completed ✓
2023-12-23 17:43:29,653 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2023-12-23 17:43:34,604 - BERTopic - Dimensionality - Completed ✓
2023-12-23 17:43:34,605 - BERTopic - Cluster - Start clustering the reduced embeddings
2023-12-23 17:43:34,912 - BERTopic - Cluster - Completed ✓
2023-12-23 17:43:34,916 - BERTopic - Representation - Extracting topics from clusters using representation models.
2023-12-23 17:43:35,025 - BERTopic - Representation - Completed ✓
100%|██████████| 1/1 [00:00<00:00, 52.44it/s]
2023-12-23 17:43:35,090 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings

---
sgns.merge.word.bz2
0.8908
---


2023-12-23 17:45:30,699 - BERTopic - Embedding - Transforming documents to embeddings.
100%|██████████| 5000/5000 [00:00<00:00, 22459.39it/s]
2023-12-23 17:45:30,946 - BERTopic - Embedding - Completed ✓
2023-12-23 17:45:30,946 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2023-12-23 17:45:35,461 - BERTopic - Dimensionality - Completed ✓
2023-12-23 17:45:35,462 - BERTopic - Cluster - Start clustering the reduced embeddings
2023-12-23 17:45:35,803 - BERTopic - Cluster - Completed ✓
2023-12-23 17:45:35,806 - BERTopic - Representation - Extracting topics from clusters using representation models.
2023-12-23 17:45:35,909 - BERTopic - Representation - Completed ✓
100%|██████████| 1/1 [00:00<00:00, 36.17it/s]
2023-12-23 17:45:35,982 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings

---
sgns.renmin.word.bz2
0.8666
---


FileNotFoundError: [Errno 2] No such file or directory: 'sgns.sogou.word.bz2sgns.target.word-word.dynwin5.thr10.neg5.dim300.iter5.bz2'

In [ ]:
topic_model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,0,1533,0_网友_出席_活动_戛纳,"[网友, 出席, 活动, 戛纳, 范冰冰, 明星, 电影, 女星, 亮相, 现身]","[邢菲 出席 活动 网友 黑 错 化妆, 张雪迎 出席 活动 网友 裙子 鞋子, 众 女..."
1,1,1282,1_手机_小米_华为_微信,"[手机, 小米, 华为, 微信, 区块, 腾讯, 苹果, 马云, 功能, 支付宝]","[华为 手机 打开 设置 手机 更加 流畅, 不止 小米 7 小米 发布 小米 8 周年纪念..."
2,2,1175,2_詹姆斯_nba_火箭_球员,"[詹姆斯, nba, 火箭, 球员, 勇士, 骑士, 球队, 保罗, 球迷, 决赛]","[NBA 本赛季 报销 球员 季后赛, 骑士 横扫 猛龙 表示 东部 决赛 轻松 数据 ..."
3,3,1010,3_中国_美国_航母_美军,"[中国, 美国, 航母, 美军, 战机, 导弹, 20, 日本, 俄罗斯, 世界]","[日本 有点 分裂 经济 需要 中国 军事 中国, 二战 国家 援助 中国 美国 外 3 国..."


In [ ]:
topic_model.get_topic(-1)

[('小米', 0.04461647808612572),
 ('农村', 0.03316040132630511),
 ('腾讯', 0.03206214066868246),
 ('没有', 0.028722944253094736),
 ('今天', 0.027648307227497945),
 ('梦想', 0.026957099546810494),
 ('支付宝', 0.022938452849435995),
 ('评价', 0.02239375314066267),
 ('导致', 0.022163528070274677),
 ('2018', 0.021828981968838986)]

In [ ]:
# 可视化选定的主题和文档
topic_model.visualize_documents(docs)


In [ ]:
topic_model.get_document_info(docs)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Representative_document
0,一个 浙江省 保险代理 公司 利用 互联网 开展业务,1,1_手机_小米_华为_微信,"[手机, 小米, 华为, 微信, 区块, 腾讯, 苹果, 马云, 功能, 支付宝]","[华为 手机 打开 设置 手机 更加 流畅, 不止 小米 7 小米 发布 小米 8 周年纪念...",手机 - 小米 - 华为 - 微信 - 区块 - 腾讯 - 苹果 - 马云 - 功能 - 支付宝,False
1,知道 2018 国际乒联 巡回赛 香港 公开赛,2,2_詹姆斯_nba_火箭_球员,"[詹姆斯, nba, 火箭, 球员, 勇士, 骑士, 球队, 保罗, 球迷, 决赛]","[NBA 本赛季 报销 球员 季后赛, 骑士 横扫 猛龙 表示 东部 决赛 轻松 数据 ...",詹姆斯 - nba - 火箭 - 球员 - 勇士 - 骑士 - 球队 - 保罗 - 球迷 - 决赛,False
2,以前 很红 火 明星 现在 没 消息 过日子,0,0_网友_出席_活动_戛纳,"[网友, 出席, 活动, 戛纳, 范冰冰, 明星, 电影, 女星, 亮相, 现身]","[邢菲 出席 活动 网友 黑 错 化妆, 张雪迎 出席 活动 网友 裙子 鞋子, 众 女...",网友 - 出席 - 活动 - 戛纳 - 范冰冰 - 明星 - 电影 - 女星 - 亮相 - 现身,False
3,当红 流量 小花 新 作品 一览 刘亦菲 杨幂 赵丽颖 作品 值得 期待,0,0_网友_出席_活动_戛纳,"[网友, 出席, 活动, 戛纳, 范冰冰, 明星, 电影, 女星, 亮相, 现身]","[邢菲 出席 活动 网友 黑 错 化妆, 张雪迎 出席 活动 网友 裙子 鞋子, 众 女...",网友 - 出席 - 活动 - 戛纳 - 范冰冰 - 明星 - 电影 - 女星 - 亮相 - 现身,False
4,印度 国家 战略 摇摆 知道 该学,3,3_中国_美国_航母_美军,"[中国, 美国, 航母, 美军, 战机, 导弹, 20, 日本, 俄罗斯, 世界]","[日本 有点 分裂 经济 需要 中国 军事 中国, 二战 国家 援助 中国 美国 外 3 国...",中国 - 美国 - 航母 - 美军 - 战机 - 导弹 - 20 - 日本 - 俄罗斯 - 世界,False
...,...,...,...,...,...,...,...
4995,旧社会 拐 妇女 血泪史,3,3_中国_美国_航母_美军,"[中国, 美国, 航母, 美军, 战机, 导弹, 20, 日本, 俄罗斯, 世界]","[日本 有点 分裂 经济 需要 中国 军事 中国, 二战 国家 援助 中国 美国 外 3 国...",中国 - 美国 - 航母 - 美军 - 战机 - 导弹 - 20 - 日本 - 俄罗斯 - 世界,False
4996,重型 龙门吊 突然 垮塌 千吨 战舰 遭 瞬间 砸烂 中国 笑笑 说话,3,3_中国_美国_航母_美军,"[中国, 美国, 航母, 美军, 战机, 导弹, 20, 日本, 俄罗斯, 世界]","[日本 有点 分裂 经济 需要 中国 军事 中国, 二战 国家 援助 中国 美国 外 3 国...",中国 - 美国 - 航母 - 美军 - 战机 - 导弹 - 20 - 日本 - 俄罗斯 - 世界,False
4997,早年 卡带 随身听 绝佳 音效 现在 找 不到,1,1_手机_小米_华为_微信,"[手机, 小米, 华为, 微信, 区块, 腾讯, 苹果, 马云, 功能, 支付宝]","[华为 手机 打开 设置 手机 更加 流畅, 不止 小米 7 小米 发布 小米 8 周年纪念...",手机 - 小米 - 华为 - 微信 - 区块 - 腾讯 - 苹果 - 马云 - 功能 - 支付宝,False
4998,范冰冰 戛纳 官方 开幕 晚宴 身着 春夏 高定 粉色 亮片 长裙,0,0_网友_出席_活动_戛纳,"[网友, 出席, 活动, 戛纳, 范冰冰, 明星, 电影, 女星, 亮相, 现身]","[邢菲 出席 活动 网友 黑 错 化妆, 张雪迎 出席 活动 网友 裙子 鞋子, 众 女...",网友 - 出席 - 活动 - 戛纳 - 范冰冰 - 明星 - 电影 - 女星 - 亮相 - 现身,False


0.8342
